## Task 1 [5] Create the University DB
#### Create the university DBMS using SQLite, a file-based DBMS.
#### Use the university DB schema provided in DDL.sql
#### Use Python (sqlite3 package) to read DDL.sql and to create the DB file.
 - Submit: createDB.py
   - No argument
   - Input filename: DDL.sql
   - Output filename : university.db

In [6]:
import sqlite3

# db 생성
con = sqlite3.connect('./university.db')
cur= con.cursor()

# sql 파일 입력 받고 command 파싱
fd = open('DDL.sql', 'r')
sqlFile = fd.read()
fd.close()
sqlCommands = sqlFile.split(';')

# 각각의 Command 실행
for command in sqlCommands:
    try:
        cur.execute(command)
    except sqlite3.Error as e:
        print("Command skipped: ", e)
        
# 만들어진 table 확인
cur.execute('SELECT name FROM sqlite_master WHERE type="table"')
row = cur.fetchall()
print("Created Table: ", len(row))
for table in row:
    print(table[0], end=', ')

# cusor와 connection 종료
con.commit()
cur.close()
con.close()

Command skipped:  table classroom already exists
Command skipped:  table department already exists
Command skipped:  table course already exists
Command skipped:  table instructor already exists
Command skipped:  table section already exists
Command skipped:  table teaches already exists
Command skipped:  table student already exists
Command skipped:  table takes already exists
Command skipped:  table advisor already exists
Command skipped:  table time_slot already exists
Command skipped:  table prereq already exists
Created Table:  11
classroom, department, course, instructor, section, teaches, student, takes, advisor, time_slot, prereq, 

## Task 2. Import actual data into the DB [10]

#### Download the CSV (comma separated value) files that contain actual data for each table (file name matches the names of DB tables).
#### Submit : insertData.py
 - No argument
 - Input files:        
 
├── data (folder)                  
│      ├── advisor.csv           
│      ├── classroom.csv         
│      ├── course.csv         
│      ├── department.csv            
│      ├── instructor.csv        
│      ├── prereq.csv          
│      ├── section.csv         
│      ├── student.csv        
│      ├── takes.csv       
│      ├── teaches.csv       
│      └── time_slot.csv        

 - Output filename : university.db (updated)

In [7]:
import sqlite3

# db 생성
con = sqlite3.connect('./university.db')
cur= con.cursor()

# 만들어진 table 확인
cur.execute('SELECT name FROM sqlite_master WHERE type="table"')
row = cur.fetchall()
print("Created Table: ", end='')
for table in row:
    print(table[0], end=', ')
print("\n")

# 각 table에 해당하는 csv 파일 읽기
for table in row:
    table=table[0]
    # 데이터 읽기
    fd = open("./data/"+table+".csv", 'r')
    data = [tuple(s.split(',')) for s in fd.read().split('\n')[1:-1]] # title 제거, tuple 형식으로 파싱
    fd.close()
    # 데이터베이스에 쓰기
    for tup in data:
        try:
            cur.execute("INSERT INTO "+table+ " VALUES "+ str(tup))
        except sqlite3.Error as e:
            print("Command skipped: ", e)

# table에 저장된 값 확인
for table in row:
    table=table[0]
    print()
    try:
        cur.execute("SELECT * FROM "+table)
        row=cur.fetchall()
        print("Table:", table, len(row))
        print(row)
    except sqlite3.Error as e:
        print("Command skipped: ", e)

# cusor와 connection 종료
con.commit()
cur.close()
con.close()


Created Table: classroom, department, course, instructor, section, teaches, student, takes, advisor, time_slot, prereq, 

Command skipped:  UNIQUE constraint failed: classroom.building, classroom.room_number
Command skipped:  UNIQUE constraint failed: classroom.building, classroom.room_number
Command skipped:  UNIQUE constraint failed: classroom.building, classroom.room_number
Command skipped:  UNIQUE constraint failed: classroom.building, classroom.room_number
Command skipped:  UNIQUE constraint failed: classroom.building, classroom.room_number
Command skipped:  UNIQUE constraint failed: department.dept_name
Command skipped:  UNIQUE constraint failed: department.dept_name
Command skipped:  UNIQUE constraint failed: department.dept_name
Command skipped:  UNIQUE constraint failed: department.dept_name
Command skipped:  UNIQUE constraint failed: department.dept_name
Command skipped:  UNIQUE constraint failed: department.dept_name
Command skipped:  UNIQUE constraint failed: department.dep

## Task 3. Queries using Python [15]

#### Make Python programs using embedded SQL to answer the following queries. 
#### query1.py 
- 학과(department)마다 소속된 강의자(instructor)들의 임금(salary)의 평균을 구하는 질의를 작성하시오. 만약 학과가 없는 강의자들이 있다면 해당 강의자(instructor)들의 임금(salary)의 평균을 구해야 합니다.

#### query2.py
- 모든 연도의 학기(semester)마다 가장 많은 과목(course)를 가르친 강의자(instructor)를 구하는 질의를 작성하시오. 만약 한 학기에 같은 수의 수업을 진행했다면 결과에 전부 포함되어야 합니다.

#### query3.py
- 2010년 summer 학기에 개설된 과목 중 오전 시간(시작 AM08:00~ 종료 PM 12:00) 사이에 개설된 과목 중 가장 많은 좌석 수(capacity)의 강의실을 사용하는 과목 명을 구하는 질의를 작성하시오. 만약 동일한 좌석 수의 과목이 있다면 결과에 전부 포함되어야 합니다.

#### condition
- No argument
- Display the resulting table onto the console.
- Print attribute names and tuples
- Attributes must be separated by a comma (,)
- Tuples must be separated by a newline (\n)

In [3]:
#query1.py
import sqlite3

# db 생성
con = sqlite3.connect('./university.db')
cur= con.cursor()

# select
try:
    cur.execute("SELECT dept_name, avg(salary) FROM instructor GROUP BY dept_name ORDER BY dept_name IS NULL")
    row=cur.fetchall()
    print('dept_name,salary')
    for tup in row:
        print(tup[0], ',', tup[1], sep='')
except sqlite3.Error as e:
    print("Command skipped: ", e)

# cusor와 connection 종료
con.commit()
cur.close()
con.close()


dept_name,salary
Biology,72000.0
Comp. Sci.,77333.33333333333
Elec. Eng.,80000.0
Finance,85000.0
History,61000.0
Music,40000.0
Physics,91000.0


In [4]:
#query2.py
import sqlite3

# db 생성
con = sqlite3.connect('./university.db')
cur= con.cursor()

#query
query='select t1.semester as semester, t1.year as year, ID from (select semester, year,count(ID) as cnt, ID from teaches group by year,semester,ID) as t1 inner join (select semester, year, max(cnt) as max_cnt from (select semester, year,count(ID) as cnt from teaches group by year,semester,ID) group by semester, year) as t2 on t1.cnt=t2.max_cnt and t1.semester=t2.semester and t1.year=t2.year'

# select
try:
    cur.execute(query)
    row=cur.fetchall()
    print('semester,year,ID')
    for tup in row:
        print(tup[0], ',', tup[1], ',', tup[2], sep='')
except sqlite3.Error as e:
    print("Command skipped: ", e)

# cusor와 connection 종료
con.commit()
cur.close()
con.close()


semester,year,ID
Fall,2009,10101
Spring,2009,83821
Summer,2009,76766
Spring,2010,45565
Summer,2010,76766


모든 연도의 학기(semester)마다 가장 많은 과목(course)를 가르친 강의자(instructor)를 구하는 질의를 작성하시오. 만약 한 학기에 같은 수의 수업을 진행했다면 결과에 전부 포함되어야 합니다.

각 학기에 대해서 count가 max인 id를 학기와 함꼐 출력
그 결과를 instructor와 조인하고 연도, 학기, id를 출력

각 학기에 교수 별로 진행한 수업의 수
select semester, year,count(ID) as cnt from teaches group by year,semester,ID;

각 학기에 가장 많은 수업을 진행한 교수가 진행한 수업의 수
select semester, year, max(cnt) as max_cnt from (select semester, year,count(ID) as cnt from teaches group by year,semester,ID) group by semester, year;

최종 결과
select t1.semester as semester, t1.year as year, ID from (select semester, year,count(ID) as cnt, ID from teaches group by year,semester,ID) as t1 inner join (select semester, year, max(cnt) as max_cnt from (select semester, year,count(ID) as cnt from teaches group by year,semester,ID) group by semester, year) as t2 on t1.cnt=t2.max_cnt and t1.semester=t2.semester and t1.year=t2.year;

In [5]:
#query3.py
import sqlite3

# db 생성
con = sqlite3.connect('./university.db')
cur= con.cursor()

#query
query="with candidate_section as (select * from (with available_time_slot_id as (select time_slot_id from (select time_slot_id, day, min(start_hr) as min_start_hr, max(end_hr) as max_end_hr from time_slot group by time_slot_id) where min_start_hr>=8 and max_end_hr<=11) select * from section where time_slot_id in available_time_slot_id and semester='Summer' and year=2010) as tmp_section natural join classroom natural join (select course_id, title from course) as course where tmp_section.building=classroom.building and tmp_section.room_number=classroom.room_number and tmp_section.course_id=course.course_id), max_capacity as (select max(capacity) as max_capacity from candidate_section) select title from candidate_section where candidate_section.capacity in max_capacity;"

# select
try:
    cur.execute(query)
    row=cur.fetchall()
    print('title')
    for tup in row:
        print(tup[0], sep='')
except sqlite3.Error as e:
    print("Command skipped: ", e)

# cusor와 connection 종료
con.commit()
cur.close()
con.close()


title
Genetics


2010년 summer 학기에 개설된 과목 중
오전 시간(시작 AM08:00~ 종료 PM 12:00) 사이에 개설된 과목 중
가장 많은 좌석 수(capacity)의 강의실을 사용하는 과목 명을 구하는 질의를 작성하시오.
만약 동일한 좌석 수의 과목이 있다면 결과에 전부 포함되어야 합니다.

- time_slot에서 start_hr가 8이상 이고 end_hr가 11이하인 것들 찾기
select time_slot_id from (select time_slot_id, day, min(start_hr) as min_start_hr, max(end_hr) as max_end_hr from time_slot group by time_slot_id) where min_start_hr>=8 and max_end_hr<=11;

- 조건에 맞는 section 찾기
with available_time_slot_id as ( select time_slot_id from (select time_slot_id, day, min(start_hr) as min_start_hr, max(end_hr) as max_end_hr from time_slot group by time_slot_id) where min_start_hr>=8 and max_end_hr<=11)
select * from section where time_slot_id in available_time_slot_id and semester='Summer' and year=2010;

- 좌석 수와 title join
with candidate_section as (
select *
from (with available_time_slot_id as (select time_slot_id from (select time_slot_id, day, min(start_hr) as min_start_hr, max(end_hr) as max_end_hr from time_slot group by time_slot_id) where min_start_hr>=8 and max_end_hr<=11)
select * from section where time_slot_id in available_time_slot_id and semester='Summer' and year=2010) as tmp_section natural join classroom natural join (select course_id, title from course) as course
where tmp_section.building=classroom.building and tmp_section.room_number=classroom.room_number and tmp_section.course_id=course.course_id)

- max capacity 찾기
with candidate_section as (
select *
from (with available_time_slot_id as (select time_slot_id from (select time_slot_id, day, min(start_hr) as min_start_hr, max(end_hr) as max_end_hr from time_slot group by time_slot_id) where min_start_hr>=8 and max_end_hr<=11)
select * from section where time_slot_id in available_time_slot_id and semester='Summer' and year=2010) as tmp_section natural join classroom natural join (select course_id, title from course) as course
where tmp_section.building=classroom.building and tmp_section.room_number=classroom.room_number and tmp_section.course_id=course.course_id), max_capacity as (
select max(capacity) as max_capacity from candidate_section)
select title from candidate_section where candidate_section.capacity in max_capacity;
